## Data Ingestion

In [10]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [11]:
# Read all pdfs inside the directory
def process_all_pdfs(pdf_directory):
    all_documents = []
    pdf_dir = Path(pdf_directory)

    pdf_files = list(pdf_dir.glob("*/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\n Processing : {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f" ✓ Loaded {len(documents)} pages")

        except Exception as e:
            print(f" ✗ Error : {e}")
    print(f"\nTotal documents loaded : {len(all_documents)}")
    return all_documents

# Process all PDFs in a data directory
all_pdf_documents = process_all_pdfs("data/")

Found 4 PDF files to process

 Processing : attention.pdf
 ✓ Loaded 15 pages

 Processing : langchain.pdf
 ✓ Loaded 75 pages

 Processing : log_linear_attention.pdf
 ✓ Loaded 21 pages

 Processing : NN_attention.pdf
 ✓ Loaded 22 pages

Total documents loaded : 133


In [18]:
# Text Splitting into Chunks

def split_documents(documents, chunk_size = 1000, chunk_overlap=200):
    """Splitting docs into smaller chunks for better RAG Performance"""
    text_spitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        seperators = ["\n\n", "\n", " ", ""]
    )
    split_docs = text_spitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata : {split_docs[0].metadata}")
    return split_docs

chunks = split_documents(all_pdf_documents)
chunks

TypeError: TextSplitter.__init__() got an unexpected keyword argument 'seperators'

In [1]:
from langchain_core.documents import Document

In [2]:
doc=Document(
    page_content="Let this be the main content I am using to create RAG",
    metadata={
        "source":"example.txt",
        "pages":1,
        "author":"shireesha",
        "date_created": "2025-12-23"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'shireesha', 'date_created': '2025-12-23'}, page_content='Let this be the main content I am using to create RAG')

In [3]:
## Create a simple txt file
import os
os.makedirs("../data/text_files", exist_ok=True)

In [4]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python is a high-level, interpreted programming language known for its clear syntax and code readability, which was created in 1991. It is popular for beginners due to its gentle learning curve and is widely used in web development, data science, artificial intelligence, and automation. 

Key features include:
Dynamic Typing: Variable types are automatically determined at runtime, which simplifies coding.
Multiple Paradigms: It supports object-oriented, functional, and procedural programming styles.
Extensive Libraries: A vast ecosystem of libraries and the Python Package Index (PyPI) offer ready-to-use code for various tasks.""",
    "data/text_files/machine_learning.txt":"""Machine Learning (ML) is a type of Artificial Intelligence (AI) that lets computers learn from data to find patterns, make decisions, and predict outcomes, without being explicitly programmed for every task, using algorithms trained on vast datasets for tasks like image recognition, recommendations, and translation. Key types include Supervised Learning (labeled data, e.g., spam filters), Unsupervised Learning (unlabeled data, e.g., customer grouping), and Reinforcement Learning (learning via rewards/penalties for decision-making). """
}

for filepath,content in sample_texts.items():
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)


print("Sample text files created!")

Sample text files created!


In [5]:
### Read the text using text loaders of langchain
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

loader.load()

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python is a high-level, interpreted programming language known for its clear syntax and code readability, which was created in 1991. It is popular for beginners due to its gentle learning curve and is widely used in web development, data science, artificial intelligence, and automation. \n\nKey features include:\nDynamic Typing: Variable types are automatically determined at runtime, which simplifies coding.\nMultiple Paradigms: It supports object-oriented, functional, and procedural programming styles.\nExtensive Libraries: A vast ecosystem of libraries and the Python Package Index (PyPI) offer ready-to-use code for various tasks.')]

In [8]:
## Directory Loader
from langchain_community.document_loaders import DirectoryLoader

dir_loader=DirectoryLoader(
    "data/text_files",
    glob="**/*.txt",
    loader_cls=TextLoader,
    show_progress=True
)

text_docs=dir_loader.load()
text_docs

100%|██████████| 2/2 [00:00<00:00, 1441.09it/s]


[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python is a high-level, interpreted programming language known for its clear syntax and code readability, which was created in 1991. It is popular for beginners due to its gentle learning curve and is widely used in web development, data science, artificial intelligence, and automation. \n\nKey features include:\nDynamic Typing: Variable types are automatically determined at runtime, which simplifies coding.\nMultiple Paradigms: It supports object-oriented, functional, and procedural programming styles.\nExtensive Libraries: A vast ecosystem of libraries and the Python Package Index (PyPI) offer ready-to-use code for various tasks.'),
 Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning (ML) is a type of Artificial Intelligence (AI) that lets computers learn from data to find patterns, make decisions, and predict outcomes, without being explicitly programmed

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader

# load all the files
dir_loader=DirectoryLoader(
    "data/pdf_files",
    glob='**/*.pdf',
    loader_cls=PyMuPDFLoader,
    show_progress=True
    )

pdf_docs=dir_loader.load()
pdf_docs


100%|██████████| 4/4 [00:00<00:00,  9.66it/s]


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'source': 'data/pdf_files/attention.pdf', 'file_path': 'data/pdf_files/attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'trapped': '', 'modDate': 'D:20240410211143Z', 'creationDate': 'D:20240410211143Z', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle

# Embedding and VectorStoreDB

In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
class EmbeddingManager:
    def __init__(self, model_name: str='all-MiniLM-L6-V2'):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model Loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model{self.model_name}:{e}")
            raise

    def generate_embeddings(self, texts:List[str])->np.ndarray:
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
        

In [6]:
# Initialize embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-V2
Model Loaded successfully. Embedding dimension: 384


# VectorStore

In [7]:
class VectorStore:
    def __init__(self, collection_name: str = "pdf_docs", persist_directory:str='data/vector_store'):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client=None
        self.collection = None
        self._initialize_store()